# COLORADO SPRINGS

In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime

# Define the file paths
input_file_path = r'C:\CMU\Capstone\CS\(Archived) COLORADO SPRINGS_ Y2_ 11_1_23-10_31_24.xlsx'
output_file_path = r'C:\CMU\Capstone\CS\Output - (Archived) COLORADO SPRINGS_ Y2_ 11_1_23-10_31_24_MonthsWorked.xlsx'

# Load the Excel file
df = pd.read_excel(input_file_path, sheet_name='(Archived) COLORADO SPRINGS, Y2')

# Define the start and end of the reporting period
report_start_date = datetime(2023, 11, 1)
report_end_date = datetime(2024, 11, 1)

# Function to calculate months worked within the report period, considering leave
def calculate_months_in_period(start_date, end_date, fte, role, name, leave_start=None, leave_end=None, include_start_month=False, skip_first_partial_month=False):
    # Adjust start date if it's before the reporting period
    if start_date < report_start_date:
        start_date = report_start_date.replace(day=1)
    elif not include_start_month:
        if start_date.day < 16:
            start_date = start_date.replace(day=1)
        else:
            start_date = start_date.replace(day=1) + relativedelta(months=1)

    # If the condition to skip the first partial month is set, advance start_date to the next-next month
    if skip_first_partial_month and start_date.day > 15:
        start_date = start_date.replace(day=1) + relativedelta(months=2)

    # Adjust end date if it's after the reporting period or other conditions
    if pd.isna(end_date):
        if pd.notna(leave_start) and pd.isna(leave_end):  # Specific case: no end date, leave start but no leave end
            end_date = leave_start - relativedelta(days=1)  # Set end date to the day before leave start
        else:
            end_date = report_end_date - relativedelta(days=1)
    elif end_date >= report_end_date:
        end_date = report_end_date - relativedelta(days=1)
    else:
        if end_date.day >= 15:
            end_date = end_date.replace(day=1) - relativedelta(days=1)
        else:
            end_date = end_date.replace(day=1) - relativedelta(months=1) - relativedelta(days=1)

    if start_date > end_date:
        return 0, 0, []  # Return 0, 0, and an empty list if the start date is after the end date

    if pd.notna(leave_start) and pd.notna(leave_end):
        if leave_end < report_start_date or leave_start > report_end_date:
            # Leave is entirely outside the reporting period
            leave_start, leave_end = None, None
        else:
            # Clip the leave period to the reporting range
            leave_start = max(leave_start, report_start_date)
            leave_end = min(leave_end, report_end_date)

    # Create a list to store each month within the period
    months_list = []
    current_date = start_date

    # Add each month from start_date to end_date into the list
    while current_date <= end_date:
        months_list.append(current_date.strftime("%Y-%m"))
        current_date += relativedelta(months=1)

    # Calculate total months worked
    total_months = len(months_list)

    # Adjust for leave period, if applicable
    if pd.notna(leave_start) and pd.notna(leave_end):
        leave_days = (leave_end - leave_start).days
        leave_months = leave_days // 22
        total_months -= leave_months

    total_months = max(0, total_months)
    fte_adjusted_months = total_months * fte

    return total_months, fte_adjusted_months, months_list


# Initialize lists to store results
months_worked_list = []
fte_adjusted_months_list = []
months_printed_list = []  # This will store the months for each entry to be printed

# Identify names with multiple entries (i.e., repeated names)
multiple_entry_names = df[df.duplicated(['Name'], keep=False)]['Name'].unique()
# Process each row in the DataFrame
for index, row in df.iterrows():
    start_date = row['Start Date']
    end_date = row['End Date'] if pd.notna(row['End Date']) else None
    fte = row['FTE'] if pd.notna(row['FTE']) else 1
    role = row['Position']
    name = row['Name']
    leave_start = row['Employee Leave (start date)']
    leave_end = row['Employee Leave (end date)']

    # Check if the entry belongs to someone with multiple entries
    multiple_entries = name in multiple_entry_names
    include_start_month = False
    skip_first_partial_month = False

    if multiple_entries:
        if pd.notna(start_date) and pd.notna(end_date):
            if start_date < report_start_date:
                start_date = report_start_date.replace(day=1)
                include_start_month = True
            else:
                if start_date.day < 15:
                    start_date = start_date.replace(day=1) + relativedelta(months=1)
                else:
                    start_date = start_date.replace(day=1) + relativedelta(months=2)

            # Handle end date
            if end_date.day < 15:
                end_date = end_date.replace(day=1) - relativedelta(days=1)
            elif end_date.month == report_start_date.month and end_date.year == report_start_date.year:
                # Append 0 values to lists for skipped rows
                months_worked_list.append(0)
                fte_adjusted_months_list.append(0)
                months_printed_list.append([])
                print(f"Skipped: {name} - {role} (End date in reporting start month)")
                continue
            else:
                end_date = end_date.replace(day=1) + relativedelta(months=1) - relativedelta(days=1)

            # Calculate months separately for this case
            months_list = []
            current_date = start_date
            while current_date <= end_date:
                months_list.append(current_date.strftime("%Y-%m"))
                current_date += relativedelta(months=1)

            months_worked = len(months_list)
            fte_adjusted_months = months_worked * fte

        elif pd.notna(start_date) and pd.isna(end_date):
            if start_date < report_start_date:
                start_date = report_start_date.replace(day=1)
                include_start_month = True
            else:
                if start_date.day < 15:
                    include_start_month = True
                else:
                    start_date = start_date.replace(day=1) + relativedelta(months=2)

            months_worked, fte_adjusted_months, months_list = calculate_months_in_period(
                start_date, end_date, fte, role, name, leave_start, leave_end, include_start_month=include_start_month, skip_first_partial_month=skip_first_partial_month
            )
    elif pd.isna(end_date) and pd.notna(leave_start) and pd.isna(leave_end):
        # Specific case: Leave start date, no end date
        if start_date < report_start_date:
            start_date = report_start_date.replace(day=1)
    
        # Adjust end date based on leave start date
        if leave_start.day < 15:
            end_date = leave_start.replace(day=1) - relativedelta(days=1)  # Last day of the previous month
        else:
            end_date = leave_start.replace(day=1) + relativedelta(months=1) - relativedelta(days=1)  # Last day of the current month

        # Calculate months worked
        months_list = []
        current_date = start_date
        while current_date <= end_date:
            months_list.append(current_date.strftime("%Y-%m"))
            current_date += relativedelta(months=1)
    
        months_worked = len(months_list)
        fte_adjusted_months = months_worked * fte
        
    else:
        if pd.notna(start_date) and start_date.day < 15:
            start_date = start_date.replace(day=1) + relativedelta(months=1)
        elif pd.notna(start_date) and start_date.day >= 15:
            start_date = start_date.replace(day=1) + relativedelta(months=2)

        months_worked, fte_adjusted_months, months_list = calculate_months_in_period(
            start_date, end_date, fte, role, name, leave_start, leave_end, include_start_month=include_start_month, skip_first_partial_month=skip_first_partial_month
        )
        
    # Ensure results are appended to lists for every row
    months_worked_list.append(months_worked)
    fte_adjusted_months_list.append(fte_adjusted_months)
    months_printed_list.append(months_list)
    print(f"Debug for {name} - {role}: Months Worked = {months_worked}, FTE = {fte}, FTE-Adjusted Months Worked = {fte_adjusted_months}")

# Add the calculated months worked and FTE-adjusted months as new columns to the DataFrame
df['Months Worked (Custom)'] = months_worked_list
df['FTE-Adjusted Months Worked (Custom)'] = fte_adjusted_months_list

# Select columns
selected_columns = [
    'Cohen Clinic',
    'Name',
    'Position',
    'FTE',
    'Start Date',
    'End Date',
    'Employee Leave (start date)',
    'Employee Leave (end date)',
    'Months Worked (Custom)',
    'FTE-Adjusted Months Worked (Custom)'    
]
filtered_df = df[selected_columns]

with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    filtered_df.to_excel(writer, sheet_name='(Archived) COLORADO SPRINGS, Y2', index=False)

print(f"Output saved to {output_file_path}")


Debug for Amista Chambers - Regional Director: Months Worked = 6, FTE = 0.3, FTE-Adjusted Months Worked = 1.7999999999999998
Debug for Amy Moore - Clinic Director: Months Worked = 12, FTE = 1.0, FTE-Adjusted Months Worked = 12.0
Debug for Andrea Keits - Intake: Months Worked = 12, FTE = 1.0, FTE-Adjusted Months Worked = 12.0
Debug for Nanci Mcdade - Office Manager: Months Worked = 12, FTE = 1.0, FTE-Adjusted Months Worked = 12.0
Debug for Bethany Wolf - Lead Clinician: Months Worked = 11, FTE = 1.0, FTE-Adjusted Months Worked = 11.0
Debug for Bethany Wolf - Clinician: Months Worked = 1, FTE = 1.0, FTE-Adjusted Months Worked = 1.0
Debug for Ellie Ruth - Clinician: Months Worked = 11, FTE = 1.0, FTE-Adjusted Months Worked = 11.0
Debug for Fitzhugh (Doug) Duggan - Front Desk/Receptionist: Months Worked = 12, FTE = 1.0, FTE-Adjusted Months Worked = 12.0
Debug for Scott Lee - Clinician: Months Worked = 12, FTE = 1.0, FTE-Adjusted Months Worked = 12.0
Debug for Matthew Stack - Outreach: Mont

# SAN ANTONIO

In [2]:
# SAN
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime

# Define the file paths
input_file_path = r'C:\CMU\Capstone\SAN ANTONIO\(Archived) SAN ANTONIO_ Y8_ 5_1_23 - 4_30_24.xlsx'
output_file_path = r'C:\CMU\Capstone\SAN ANTONIO\Output - (Archived) SAN ANTONIO_ Y8_ 5_1_23 - 4_30_24_MonthsWorked.xlsx'

# Load the Excel file
df = pd.read_excel(input_file_path, sheet_name='(Archived) SAN ANTONIO, Y8, 5_1')

# Define the start and end of the reporting period
report_start_date = datetime(2023, 5, 1)
report_end_date = datetime(2024, 5, 1)

# Function to calculate months worked within the report period, considering leave
def calculate_months_in_period(start_date, end_date, fte, role, name, leave_start=None, leave_end=None, include_start_month=False, skip_first_partial_month=False):
    # Adjust start date if it's before the reporting period
    if start_date < report_start_date:
        start_date = report_start_date.replace(day=1)
    elif not include_start_month:
        if start_date.day < 16:
            start_date = start_date.replace(day=1)
        else:
            start_date = start_date.replace(day=1) + relativedelta(months=1)

    # If the condition to skip the first partial month is set, advance start_date to the next-next month
    if skip_first_partial_month and start_date.day > 15:
        start_date = start_date.replace(day=1) + relativedelta(months=2)

    # Adjust end date if it's after the reporting period or other conditions
    if pd.isna(end_date):
        if pd.notna(leave_start) and pd.isna(leave_end):  # Specific case: no end date, leave start but no leave end
            end_date = leave_start - relativedelta(days=1)  # Set end date to the day before leave start
        else:
            end_date = report_end_date - relativedelta(days=1)
    elif end_date >= report_end_date:
        end_date = report_end_date - relativedelta(days=1)
    else:
        if end_date.day >= 15:
            end_date = end_date.replace(day=1) - relativedelta(days=1)
        else:
            end_date = end_date.replace(day=1) - relativedelta(months=1) - relativedelta(days=1)

    if start_date > end_date:
        return 0, 0, []  # Return 0, 0, and an empty list if the start date is after the end date

    if pd.notna(leave_start) and pd.notna(leave_end):
        if leave_end < report_start_date or leave_start > report_end_date:
            # Leave is entirely outside the reporting period
            leave_start, leave_end = None, None
        else:
            # Clip the leave period to the reporting range
            leave_start = max(leave_start, report_start_date)
            leave_end = min(leave_end, report_end_date)

    # Create a list to store each month within the period
    months_list = []
    current_date = start_date

    # Add each month from start_date to end_date into the list
    while current_date <= end_date:
        months_list.append(current_date.strftime("%Y-%m"))
        current_date += relativedelta(months=1)

    # Calculate total months worked
    total_months = len(months_list)

    # Adjust for leave period, if applicable
    if pd.notna(leave_start) and pd.notna(leave_end):
        leave_days = (leave_end - leave_start).days
        leave_months = leave_days // 22
        total_months -= leave_months

    total_months = max(0, total_months)
    fte_adjusted_months = total_months * fte

    return total_months, fte_adjusted_months, months_list

# Initialize lists to store results
months_worked_list = []
fte_adjusted_months_list = []
months_printed_list = []  # This will store the months for each entry to be printed

# Identify names with multiple entries (i.e., repeated names)
multiple_entry_names = df[df.duplicated(['Name'], keep=False)]['Name'].unique()
# Process each row in the DataFrame
for index, row in df.iterrows():
    start_date = row['Start Date']
    end_date = row['End Date'] if pd.notna(row['End Date']) else None
    fte = row['FTE'] if pd.notna(row['FTE']) else 1
    role = row['Position']
    name = row['Name']
    leave_start = row['Employee Leave (start date)']
    leave_end = row['Employee Leave (end date)']

    # Check if the entry belongs to someone with multiple entries
    multiple_entries = name in multiple_entry_names
    include_start_month = False
    skip_first_partial_month = False

    if multiple_entries:
        if pd.notna(start_date) and pd.notna(end_date):
            if start_date < report_start_date:
                start_date = report_start_date.replace(day=1)
                include_start_month = True
            else:
                if start_date.day < 15:
                    start_date = start_date.replace(day=1) + relativedelta(months=1)
                else:
                    start_date = start_date.replace(day=1) + relativedelta(months=2)

            # Handle end date
            if end_date.day < 15:
                end_date = end_date.replace(day=1) - relativedelta(days=1)
            elif end_date.month == report_start_date.month and end_date.year == report_start_date.year:
                # Append 0 values to lists for skipped rows
                months_worked_list.append(0)
                fte_adjusted_months_list.append(0)
                months_printed_list.append([])
                print(f"Skipped: {name} - {role} (End date in reporting start month)")
                continue
            else:
                end_date = end_date.replace(day=1) + relativedelta(months=1) - relativedelta(days=1)

            # Calculate months separately for this case
            months_list = []
            current_date = start_date
            while current_date <= end_date:
                months_list.append(current_date.strftime("%Y-%m"))
                current_date += relativedelta(months=1)

            months_worked = len(months_list)
            fte_adjusted_months = months_worked * fte

        elif pd.notna(start_date) and pd.isna(end_date):
            if start_date < report_start_date:
                start_date = report_start_date.replace(day=1)
                include_start_month = True
            else:
                if start_date.day < 15:
                    include_start_month = True
                else:
                    start_date = start_date.replace(day=1) + relativedelta(months=2)

            months_worked, fte_adjusted_months, months_list = calculate_months_in_period(
                start_date, end_date, fte, role, name, leave_start, leave_end, include_start_month=include_start_month, skip_first_partial_month=skip_first_partial_month
            )
    elif pd.isna(end_date) and pd.notna(leave_start) and pd.isna(leave_end):
        # Specific case: Leave start date, no end date
        if start_date < report_start_date:
            start_date = report_start_date.replace(day=1)
    
        # Adjust end date based on leave start date
        if leave_start.day < 15:
            end_date = leave_start.replace(day=1) - relativedelta(days=1)  # Last day of the previous month
        else:
            end_date = leave_start.replace(day=1) + relativedelta(months=1) - relativedelta(days=1)  # Last day of the current month

        # Calculate months worked
        months_list = []
        current_date = start_date
        while current_date <= end_date:
            months_list.append(current_date.strftime("%Y-%m"))
            current_date += relativedelta(months=1)
    
        months_worked = len(months_list)
        fte_adjusted_months = months_worked * fte
        
    else:
        if pd.notna(start_date) and start_date.day < 15:
            start_date = start_date.replace(day=1) + relativedelta(months=1)
        elif pd.notna(start_date) and start_date.day >= 15:
            start_date = start_date.replace(day=1) + relativedelta(months=2)

        months_worked, fte_adjusted_months, months_list = calculate_months_in_period(
            start_date, end_date, fte, role, name, leave_start, leave_end, include_start_month=include_start_month, skip_first_partial_month=skip_first_partial_month
        )
        
    # Ensure results are appended to lists for every row
    months_worked_list.append(months_worked)
    fte_adjusted_months_list.append(fte_adjusted_months)
    months_printed_list.append(months_list)
    print(f"Debug for {name} - {role}: Months Worked = {months_worked}, FTE = {fte}, FTE-Adjusted Months Worked = {fte_adjusted_months}")

# Add the calculated months worked and FTE-adjusted months as new columns to the DataFrame
df['Months Worked (Custom)'] = months_worked_list
df['FTE-Adjusted Months Worked (Custom)'] = fte_adjusted_months_list

# Select columns
selected_columns = [
    'Cohen Clinic',
    'Name',
    'Position',
    'FTE',
    'Start Date',
    'End Date',
    'Employee Leave (start date)',
    'Employee Leave (end date)',
    'Months Worked (Custom)',
    'FTE-Adjusted Months Worked (Custom)'    
]
filtered_df = df[selected_columns]

# Save the updated DataFrame to a new Excel file with custom columns
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    filtered_df.to_excel(writer, sheet_name='(Archived) SAN ANTONIO, Y8, 5_1', index=False)

print(f"Output saved to {output_file_path}")


Debug for Laura Knoll - Clinician: Months Worked = 0, FTE = 1.0, FTE-Adjusted Months Worked = 0
Debug for Christen Greenfield - Intern: Months Worked = 0, FTE = 0.25, FTE-Adjusted Months Worked = 0
Skipped: Rhonda Gaston - Lead Clinician (End date in reporting start month)
Debug for Lisa Sierra - Intern: Months Worked = 0, FTE = 0.25, FTE-Adjusted Months Worked = 0
Debug for Carolina Echeverria - Case Management: Months Worked = 0, FTE = 0.5, FTE-Adjusted Months Worked = 0
Debug for Jeanette James - Clinician: Months Worked = 1, FTE = 1.0, FTE-Adjusted Months Worked = 1.0
Debug for Ashley Brand - Intern: Months Worked = 0, FTE = 0.25, FTE-Adjusted Months Worked = 0
Debug for Melissa Baker - Fellow: Months Worked = 2, FTE = 1.0, FTE-Adjusted Months Worked = 2.0
Debug for Holly King - Intern: Months Worked = 0, FTE = 0.25, FTE-Adjusted Months Worked = 0
Debug for Frantz Moiseau - Intern: Months Worked = 2, FTE = 0.25, FTE-Adjusted Months Worked = 0.5
Debug for Wendy Van Dyke - Intern: Mo

C:\Users\ashpp\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
